In [178]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

# Tworzenie tabeli z zebranymi ogłoszeniami.

In [195]:
def poczatek():
    url1 = input('Wprowadź link do strony z wynikami Twojego wyszukiwania. Przejdę przez wszystkie dostępne strony. ')
    strona = requests.get(url1)
    soup = bs(strona.text,'lxml')
    
    return soup, url1

#soup, url = poczatek()

In [180]:
soup, url1 = poczatek()

Wprowadź link do strony z wynikami Twojego wyszukiwaniahttps://www.otodom.pl/wynajem/mieszkanie/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bfilter_enum_rooms_num%5D%5B0%5D=2&search%5Bfilter_enum_rooms_num%5D%5B1%5D=3&search%5Bfilter_enum_rooms_num%5D%5B2%5D=1&locations%5B0%5D%5Bregion_id%5D=12&locations%5B0%5D%5Bsubregion_id%5D=294&locations%5B0%5D%5Bcity_id%5D=166&locations%5B1%5D%5Bregion_id%5D=12&locations%5B1%5D%5Bsubregion_id%5D=315&locations%5B1%5D%5Bcity_id%5D=165


In [181]:
url1

'https://www.otodom.pl/wynajem/mieszkanie/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bfilter_enum_rooms_num%5D%5B0%5D=2&search%5Bfilter_enum_rooms_num%5D%5B1%5D=3&search%5Bfilter_enum_rooms_num%5D%5B2%5D=1&locations%5B0%5D%5Bregion_id%5D=12&locations%5B0%5D%5Bsubregion_id%5D=294&locations%5B0%5D%5Bcity_id%5D=166&locations%5B1%5D%5Bregion_id%5D=12&locations%5B1%5D%5Bsubregion_id%5D=315&locations%5B1%5D%5Bcity_id%5D=165'

In [182]:
soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Mieszkanie  na wynajem w Tychy; Zabrze - www.otodom.pl</title>
<link href="https://www.otodom.pl/wynajem/mieszkanie/?search%5Bfilter_float_m%3Ato%5D=50&amp;search%5Bfilter_enum_rooms_num%5D%5B0%5D=2&amp;search%5Bfilter_enum_rooms_num%5D%5B1%5D=3&amp;search%5Bfilter_enum_rooms_num%5D%5B2%5D=1&amp;locations%5B0%5D%5Bregion_id%5D=12&amp;locations%5B0%5D%5Bsubregion_id%5D=294&amp;locations%5B0%5D%5Bcity_id%5D=166&amp;locations%5B1%5D%5Bregion_id%5D=12&amp;locations%5B1%5D%5Bsubregion_id%5D=315&amp;locations%5B1%5D%5Bcity_id%5D=165&amp;page=2" rel="next"/>
<meta content="index, follow" name="robots"/> <link href="https://www.otodom.pl/wynajem/mieszkanie/" rel="canonical"/> <meta content="pl" http-equiv="Content-Language"/>
<meta content="Zobacz 107 mieszkań do wynajęcia z co najmniej 2, 3, 1 pokojami w miejscowośc

In [183]:
def zakres():    
    p2 = soup.find('form', {"id":'pagerForm'})
    p2 = p2.find_all('a',"")

    strony = []

    for a in p2:
        strona = a.text
        if strona != "":
            strona = int(strona)
            strony.append(strona)
        else:
            pass
    strona_max = max(strony)
    strona_min = min(strony)

    return(strona_min, strona_max)

#strona_min, strona_max = zakres()

In [184]:
strona_min,strona_max = zakres()

In [185]:
strona_min

1

In [186]:
strona_max

5

In [187]:
def limit_wyszukiwan(strona_max):

    limit_wyszukiwania = 0
    
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    
    return limit_wyszukiwania

In [188]:
limit_wyszukiwania = limit_wyszukiwan(strona_max)
limit_wyszukiwania

5

In [205]:
def parametry():
    
    #tworze główny data frame
    df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena'])
    
    #listy na wartości z ogłoszeń + tworzenie kolejnych iteracji linku
    for i in range(strona_min,limit_wyszukiwania+1):
        
        l_tytul = []
        l_typ = []
        l_miejsce = []
        l_pokoje = []
        l_powierzchnia = []
        l_cena = []
        l_wynajmujacy = []
        ##lista_list = [l_tytul,l_typ,l_miejsce,l_pokoje,l_powierzchnia,l_cena,l_wynajmujacy]
        
        url = url1+ f'&page={i}'
        strona = requests.get(url)
        soup = bs(strona.text,'lxml')

        oferty = soup.find_all('article')
    
        #pętla ściągania ofert z pojedynczej strony wyszukiwania
        for oferta in oferty:

            tytul = oferta.find('span', {"class":"offer-item-title"}).text
            l_tytul.append(tytul)

            typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
            l_typ.append(typ)

            miejsce = oferta.find('p', {"class": "text-nowrap"}).text
            miejsce = miejsce[miejsce.find(":",0,-1)+2:]
            l_miejsce.append(miejsce)

            pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
            l_pokoje.append(pokoje)

            powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
            l_powierzchnia.append(powierzchnia)

            cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
            l_cena.append(cena)

            wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
            l_wynajmujacy.append(wynajmujacy)

        
        #tworze data frame z jednej strony wyszukiwania
        df1 = pd.DataFrame({'tytul':l_tytul, 
                                 "typ":l_typ,
                                 "miejsce": l_miejsce,
                                 "pokoje": l_pokoje,
                                 "powierzchnia": l_powierzchnia,
                                 "cena": l_cena,
                                 "wynajmujacy": l_wynajmujacy}) 
        
        #dodaję df z jednej strony wyszukiwania do df głównego
        ramki = [df,df1]
        df = pd.concat(ramki)
    
    
    #dopasowuje automatycznie typy danych
    df = df.convert_dtypes()
    
    #dodaje typy danych kategoryczne
    df['miasto'] = df['miejsce'].str.split(',').str[0]
    df['miasto'] = df['miasto'].astype('category')
    df['typ'] = df['typ'].astype('category')
    df['miejsce'] = df['miejsce'].astype('category')
    df['wynajmujacy'] = df['wynajmujacy'].astype('category')
    
    #zmiana kolejności kolumn
    df = df[['tytul','typ','miasto','miejsce','pokoje','powierzchnia','cena','wynajmujacy']]
    
    #usuwanie duplikatów wierszy
    df = df.drop_duplicates()
    
    return df

In [203]:
tabela = parametry()

In [204]:
tabela

,tytul,typ,miasto,miejsce,pokoje,powierzchnia,cena,wynajmujacy
0,Dwupokojowy apartament w centrum Zabrza,Mieszkanie na wynajem,Zabrze,"Zabrze, Centrum",2,40,1400,Oferta prywatna
1,Mieszkanie Zabrze os. Janek,Mieszkanie na wynajem,Zabrze,"Zabrze, Centrum",2,38,1500,Oferta prywatna
2,Ładna kawalerka umeblowana i wyposażona,Mieszkanie na wynajem,Zabrze,"Zabrze, Centrum",1,25,1100,Obsługa zdalna\n\n ...
3,Zabrze centrum,Mieszkanie na wynajem,Zabrze,"Zabrze, Centrum",1,34,1100,Oferta prywatna
4,Wynajmę eleganckie mieszkanie dwupokojowe,Mieszkanie na wynajem,Tychy,"Tychy, C",2,39,1500,Obsługa zdalna\n\n ...
...,...,...,...,...,...,...,...,...
9,Wynajem 2 37 mkw pokoje Morcinka,Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,37,1500,JWPach Nieruchomosci
10,"Mieszkanie, 37 m², Tychy",Mieszkanie na wynajem,Tychy,"Tychy, H",2,37,1450,Kulańska Nieruchomości
11,"Mieszkanie, 36 m², Tychy",Mieszkanie na wynajem,Tychy,"Tychy, Urbanowice",1,36,1590,Kulańska Nieruchomości
12,"Mieszkanie, 39,36 m², Tychy",Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,39,1700,H.B.K Nieruchomości


In [194]:
tabela.dtypes

tytul             string
typ             category
miasto          category
miejsce         category
pokoje             Int64
powierzchnia       Int64
cena               Int64
wynajmujacy     category
dtype: object

# Analiza i tabele

In [210]:
#podsumowanie zebranych ofert

def podsumowanie_ofert():
    
    #podsu = tabela.groupby(['miasto','pokoje']).count().reset_index()
    
    podsu = tabela.groupby(['miasto','pokoje']).agg({'cena':['mean', 'max', 'min'], 
                         'powierzchnia':'mean', 
                         'tytul':'count'})
    
    print("Podsumowanie ofert z Twojego wyszukiwania OTODOM: ")
    
    return podsu

In [212]:
podsumowanie = podsumowanie_ofert()
podsumowanie

Podsumowanie ofert z Twojego wyszukiwania OTODOM: 


cena              powierzchnia tytul
                      mean    max   min         mean count
miasto pokoje                                             
Tychy  1       1362.500000   1590  1200    32.875000     8
       2       2110.575758  14800  1400    40.484848    33
       3        500.000000    500   500    42.000000     1
Zabrze 1       1128.500000   1500   750    29.333333    24
       2       1342.459459   1800   900    38.270270    37
       3       1450.000000   1450  1450    44.000000     1